In [1]:
import yaml
import csv
from collections import defaultdict

In [3]:
file_path_yaml = r'd:\CodeRun\table_statistics\data\sample.yaml'

In [4]:
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)